In [78]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

In [79]:
df=pd.read_csv('train_data.csv')


In [136]:
X=np.array(df.drop(['y','datetime'],axis=1).values)
y=np.array(df['y'].values)
X=X.reshape(X.shape[0],X.shape[1],1)
X_train,X_test,y_train,y_test=train_test_split(X,y,shuffle=False)
#X_train

In [82]:
model_save_path = './pose_test.h5'

In [170]:
length_of_sequence = X_train.shape[1] 
in_out_neurons = X_train.shape[0]
n_hidden = 5

model = Sequential()
model.add(LSTM(64, return_sequences=True,activation='relu', input_shape= (X_train.shape[1],X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(Dropout(0.5))
model.add(LSTM(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1,activation='linear'))

model.summary()
#model.weights

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_72 (LSTM)               (None, 11, 64)            16896     
_________________________________________________________________
dropout_54 (Dropout)         (None, 11, 64)            0         
_________________________________________________________________
lstm_73 (LSTM)               (None, 11, 128)           98816     
_________________________________________________________________
dropout_55 (Dropout)         (None, 11, 128)           0         
_________________________________________________________________
lstm_74 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dropout_56 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 64)              

In [171]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [172]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='mean_absolute_error',
    metrics=['mae']
)

In [173]:
early_stopping = EarlyStopping(monitor='val_loss', mode='auto', patience=20)
# 学習
model.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/30
12/13 [==========================>...] - ETA: 0s - loss: 30.0965 - mae: 30.0965
Epoch 00001: saving model to ./pose_test.h5
13/13 [==============================] - 2s 187ms/step - loss: 30.1155 - mae: 30.1155 - val_loss: 44.8934 - val_mae: 44.8934
Epoch 2/30
12/13 [==========================>...] - ETA: 0s - loss: 24.6710 - mae: 24.6710
Epoch 00002: saving model to ./pose_test.h5
13/13 [==============================] - 1s 93ms/step - loss: 24.6443 - mae: 24.6443 - val_loss: 21.5217 - val_mae: 21.5217
Epoch 3/30
13/13 [==============================] - ETA: 0s - loss: 13.0156 - mae: 13.0156
Epoch 00003: saving model to ./pose_test.h5
13/13 [==============================] - 1s 108ms/step - loss: 13.0156 - mae: 13.0156 - val_loss: 21.8045 - val_mae: 21.8045
Epoch 4/30
12/13 [==========================>...] - ETA: 0s - loss: 11.4761 - mae: 11.4761
Epoch 00004: saving model to ./pose_test.h5
13/13 [==============================] - 2s 130ms/step - loss: 11.4757 - mae: 11.4757 

In [174]:
model.load_weights('./pose_test.h5')
# 評価
#from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from sklearn.metrics import mean_absolute_error
y_pred = model.predict(X_test)


loss=mean_absolute_error(y_pred,y_test)
loss


16.513472259724548

In [64]:
test_=pd.read_csv('test_data.csv')

In [65]:
test=np.array(test_.drop('datetime',axis=1))
test=X_test.reshape(test.shape[0],X_test.shape[1],1)

In [66]:
y_test=model.predict(test)

In [73]:
csv=pd.DataFrame(list(y_test[:,0]),test_['datetime'])
csv.to_csv('提出csv/LSTM.csv')

In [72]:
ls

LSTM.ipynb                    train_data.csv
SVR.ipynb                     xGboostに突っ込む.ipynb
pose_test.h5                  提出csv/
sample_submit.csv             前処理.ipynb
test.csv                      特徴量抽出.ipynb
test_data.csv                 データ可視化.ipynb
train.csv
